In [3]:
import torch
import torch.nn as nn
from torch.nn import init
import torch.nn.functional as F
from torch.autograd import Variable
import pickle
import numpy as np
import time
import random
from collections import defaultdict
import torch.utils.data
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
import datetime
import argparse
import os
from tqdm import tqdm
import pandas as pd

In [4]:
class Attention(nn.Module):
    def __init__(self, embedding_dims):
        super(Attention, self).__init__()
        self.embed_dim = embedding_dims
        self.bilinear = nn.Bilinear(self.embed_dim, self.embed_dim, 1)
        self.att1 = nn.Linear(self.embed_dim * 2, self.embed_dim)
        self.att2 = nn.Linear(self.embed_dim, self.embed_dim)
        self.att3 = nn.Linear(self.embed_dim, 1)
        self.softmax = nn.Softmax(0)

    def forward(self, node1, u_rep, num_neighs):
        uv_reps = u_rep.repeat(num_neighs, 1)
        x = torch.cat((node1, uv_reps), 1)
        x = F.relu(self.att1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.att2(x))
        x = F.dropout(x, training=self.training)
        x = self.att3(x)
        att = F.softmax(x, dim=0)
        return att

class Social_Encoder(nn.Module):

    def __init__(self, features, embed_dim, social_adj_lists, aggregator, base_model=None, cuda="cpu"):
        super(Social_Encoder, self).__init__()

        self.features = features
        self.social_adj_lists = social_adj_lists
        self.aggregator = aggregator
        if base_model != None:
            self.base_model = base_model
        self.embed_dim = embed_dim
        self.device = cuda
        self.linear1 = nn.Linear(2 * self.embed_dim, self.embed_dim)  #

    def forward(self, nodes):

        to_neighs = []
        for node in nodes:
            # to_neighs.append(self.social_adj_lists[int(node)])
            to_neighs.append(self.social_adj_lists.get(int(node), set())) # Use .get() with default empty set for missing users
        neigh_feats = self.aggregator.forward(nodes, to_neighs)  # user-user network

        self_feats = self.features(torch.LongTensor(nodes.cpu().numpy())).to(self.device)
        self_feats = self_feats.t()

        # self-connection could be considered.
        combined = torch.cat([self_feats, neigh_feats], dim=1)
        combined = F.relu(self.linear1(combined))

        return combined

class Social_Aggregator(nn.Module):
    """
    Social Aggregator: for aggregating embeddings of social neighbors.
    """

    def __init__(self, features, u2e, embed_dim, cuda="cpu"):
        super(Social_Aggregator, self).__init__()

        self.features = features
        self.device = cuda
        self.u2e = u2e
        self.embed_dim = embed_dim
        self.att = Attention(self.embed_dim)

    def forward(self, nodes, to_neighs):
        embed_matrix = torch.empty(len(nodes), self.embed_dim, dtype=torch.float).to(self.device)
        for i in range(len(nodes)):
            tmp_adj = to_neighs[i]
            num_neighs = len(tmp_adj)
            #
            e_u = self.u2e.weight[list(tmp_adj)] # fast: user embedding
            #slow: item-space user latent factor (item aggregation)
            #feature_neigbhors = self.features(torch.LongTensor(list(tmp_adj)).to(self.device))
            #e_u = torch.t(feature_neigbhors)

            u_rep = self.u2e.weight[nodes[i]]

            att_w = self.att(e_u, u_rep, num_neighs)
            att_history = torch.mm(e_u.t(), att_w).t()
            embed_matrix[i] = att_history
        to_feats = embed_matrix

        return to_feats

class UV_Aggregator(nn.Module):
    """
    item and user aggregator: for aggregating embeddings of neighbors (item/user aggreagator).
    """

    def __init__(self, v2e, r2e, u2e, embed_dim, cuda="cpu", uv=True):
        super(UV_Aggregator, self).__init__()
        self.uv = uv
        self.v2e = v2e
        self.r2e = r2e
        self.u2e = u2e
        self.device = cuda
        self.embed_dim = embed_dim
        self.w_r1 = nn.Linear(self.embed_dim * 2, self.embed_dim)
        self.w_r2 = nn.Linear(self.embed_dim, self.embed_dim)
        self.att = Attention(self.embed_dim)

    def forward(self, nodes, history_uv, history_r):

        embed_matrix = torch.empty(len(history_uv), self.embed_dim, dtype=torch.float).to(self.device)

        for i in range(len(history_uv)):
            history = history_uv[i]
            num_histroy_item = len(history)
            tmp_label = history_r[i]

            if self.uv == True:
                # user component
                e_uv = self.v2e.weight[history]
                uv_rep = self.u2e.weight[nodes[i]]
            else:
                # item component
                e_uv = self.u2e.weight[history]
                uv_rep = self.v2e.weight[nodes[i]]

            e_r = self.r2e.weight[tmp_label]
            x = torch.cat((e_uv, e_r), 1)
            x = F.relu(self.w_r1(x))
            o_history = F.relu(self.w_r2(x))

            att_w = self.att(o_history, uv_rep, num_histroy_item)
            att_history = torch.mm(o_history.t(), att_w)
            att_history = att_history.t()

            embed_matrix[i] = att_history
        to_feats = embed_matrix
        return to_feats

class UV_Encoder(nn.Module):

    def __init__(self, features, embed_dim, history_uv_lists, history_r_lists, aggregator, cuda="cpu", uv=True):
        super(UV_Encoder, self).__init__()

        self.features = features
        self.uv = uv
        self.history_uv_lists = history_uv_lists
        self.history_r_lists = history_r_lists
        self.aggregator = aggregator
        self.embed_dim = embed_dim
        self.device = cuda
        self.linear1 = nn.Linear(2 * self.embed_dim, self.embed_dim)  #

    def forward(self, nodes):
        tmp_history_uv = []
        tmp_history_r = []
        for node in nodes:
            tmp_history_uv.append(self.history_uv_lists[int(node)])
            tmp_history_r.append(self.history_r_lists[int(node)])

        neigh_feats = self.aggregator.forward(nodes, tmp_history_uv, tmp_history_r)  # user-item network

        self_feats = self.features.weight[nodes]
        # self-connection could be considered.
        combined = torch.cat([self_feats, neigh_feats], dim=1)
        combined = F.relu(self.linear1(combined))

        return combined

class GraphRec(nn.Module):

    def __init__(self, enc_u, enc_v_history, r2e):
        super(GraphRec, self).__init__()
        self.enc_u = enc_u
        self.enc_v_history = enc_v_history
        self.embed_dim = enc_u.embed_dim

        self.w_ur1 = nn.Linear(self.embed_dim, self.embed_dim)
        self.w_ur2 = nn.Linear(self.embed_dim, self.embed_dim)
        self.w_vr1 = nn.Linear(self.embed_dim, self.embed_dim)
        self.w_vr2 = nn.Linear(self.embed_dim, self.embed_dim)
        self.w_uv1 = nn.Linear(self.embed_dim * 2, self.embed_dim)
        self.w_uv2 = nn.Linear(self.embed_dim, 16)
        self.w_uv3 = nn.Linear(16, 1)
        self.r2e = r2e
        self.bn1 = nn.BatchNorm1d(self.embed_dim, momentum=0.5)
        self.bn2 = nn.BatchNorm1d(self.embed_dim, momentum=0.5)
        self.bn3 = nn.BatchNorm1d(self.embed_dim, momentum=0.5)
        self.bn4 = nn.BatchNorm1d(16, momentum=0.5)
        self.criterion = nn.MSELoss()

    def forward(self, nodes_u, nodes_v):
        embeds_u = self.enc_u(nodes_u)
        embeds_v = self.enc_v_history(nodes_v)

        x_u = F.relu(self.bn1(self.w_ur1(embeds_u)))
        x_u = F.dropout(x_u, training=self.training)
        x_u = self.w_ur2(x_u)
        x_v = F.relu(self.bn2(self.w_vr1(embeds_v)))
        x_v = F.dropout(x_v, training=self.training)
        x_v = self.w_vr2(x_v)

        x_uv = torch.cat((x_u, x_v), 1)
        x = F.relu(self.bn3(self.w_uv1(x_uv)))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.bn4(self.w_uv2(x)))
        x = F.dropout(x, training=self.training)
        scores = self.w_uv3(x)
        return scores.squeeze()

    def loss(self, nodes_u, nodes_v, labels_list):
        scores = self.forward(nodes_u, nodes_v)
        return self.criterion(scores, labels_list)

In [5]:
def initialize_and_load_model(checkpoint_path, num_users, num_items, num_ratings, embed_dim, history_u_lists, history_ur_lists, history_v_lists, history_vr_lists, social_adj_lists, device):
    """
    Initialize model architecture and load saved weights
    Args:
        checkpoint_path: Path to the saved checkpoint
        num_users: Number of users in the dataset
        num_items: Number of items in the dataset
        num_ratings: Number of possible ratings
        embed_dim: Embedding dimension
        history_u_lists: User history lists
        history_ur_lists: User rating history lists
        history_v_lists: Item history lists
        history_vr_lists: Item rating history lists
        social_adj_lists: Social adjacency lists
        device: Device to run model on
    Returns:
        model: Loaded model ready for inference
        epoch: The epoch number of the checkpoint
        rmse: The RMSE score of the checkpoint
        mae: The MAE score of the checkpoint
    """
    # Initialize embeddings
    u2e = nn.Embedding(num_users, embed_dim).to(device)
    v2e = nn.Embedding(num_items, embed_dim).to(device)
    r2e = nn.Embedding(num_ratings, embed_dim).to(device)

    # Initialize user feature components
    agg_u_history = UV_Aggregator(v2e, r2e, u2e, embed_dim, cuda=device, uv=True)
    enc_u_history = UV_Encoder(u2e, embed_dim, history_u_lists, history_ur_lists, agg_u_history, cuda=device, uv=True)
    agg_u_social = Social_Aggregator(lambda nodes: enc_u_history(nodes).t(), u2e, embed_dim, cuda=device)
    enc_u = Social_Encoder(lambda nodes: enc_u_history(nodes).t(), embed_dim, social_adj_lists, agg_u_social,
                          base_model=enc_u_history, cuda=device)

    # Initialize item feature components
    agg_v_history = UV_Aggregator(v2e, r2e, u2e, embed_dim, cuda=device, uv=False)
    enc_v_history = UV_Encoder(v2e, embed_dim, history_v_lists, history_vr_lists, agg_v_history, cuda=device, uv=False)

    # Initialize the model
    model = GraphRec(enc_u, enc_v_history, r2e).to(device)

    # Load the saved weights
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    epoch = checkpoint['epoch']
    rmse = checkpoint['rmse']
    mae = checkpoint['mae']

    return model, epoch, rmse, mae

In [6]:
def load_model(checkpoint_path, model, optimizer=None):
    """
    Load a saved model checkpoint
    Args:
        checkpoint_path: Path to the saved checkpoint
        model: The model instance to load weights into
        optimizer: Optional optimizer to load state
    Returns:
        model: Loaded model
        optimizer: Loaded optimizer (if provided)
        epoch: The epoch number of the checkpoint
        rmse: The RMSE score of the checkpoint
        mae: The MAE score of the checkpoint
    """
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    rmse = checkpoint['rmse']
    mae = checkpoint['mae']
    return model, optimizer, epoch, rmse, mae

In [7]:
def create_result(user_ids, item_ids, predicted_ratings_continuous, predicted_ratings_round, predicted_ratings, key_rating, value_rating, rmse, mae):
    # Create a DataFrame
    data = {
        'user_id': user_ids,
        'item': item_ids,
        'predicted_rating_continuous': predicted_ratings_continuous,
        'predicted_rating_round': predicted_ratings_round,
        'target_key': key_rating,
        'predicted_rating': predicted_ratings,
        'target_label': value_rating
    }

    df = pd.DataFrame(data)

    # Create the result directory if it doesn't exist
    result_dir = 'result'
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)
    
    # Export metrics to a text file
    metrics_file_path = os.path.join(result_dir, 'metrics.txt')
    with open(metrics_file_path, 'w') as f:
        f.write(f"RMSE: {rmse:.4f}\n")
        f.write(f"MAE: {mae:.4f}\n")
    print(f"Metrics exported to {metrics_file_path}")

    # Specify the path for the CSV file
    csv_file_path = os.path.join(result_dir, 'predictions.csv')

    # Export to CSV
    df.to_csv(csv_file_path, index=False)

    print(f"Data exported to {csv_file_path}")

In [8]:
def test(model, device, test_loader, ratings_list):
    model.eval()
    user_id_list = []
    item_id_list = []
    tmp_pred = []
    target = []
    predictions = []
    rounded_ratings = []
    actual_targets = []
    with torch.no_grad():
        for test_u, test_v, tmp_target in tqdm(test_loader, desc="Testing"):
            test_u, test_v, tmp_target = test_u.to(device), test_v.to(device), tmp_target.to(device)
            val_output = model.forward(test_u, test_v)
            user_id_list.extend(test_u.tolist())
            item_id_list.extend(test_v.tolist())
            tmp_pred.append(list(val_output.data.cpu().numpy()))
            target.append(list(tmp_target.data.cpu().numpy()))
    tmp_pred = np.array(sum(tmp_pred, []))
    target = np.array(sum(target, []))
    expected_rmse = sqrt(mean_squared_error(tmp_pred, target))
    mae = mean_absolute_error(tmp_pred, target)

    # Convert the continuous value to the nearest key of ratings_list
    for i in range(len(tmp_pred)):
        predicted_rating = tmp_pred[i]
        rounded_rating = min(ratings_list.keys(), key=lambda x: abs(x - predicted_rating))
        rounded_ratings.append(rounded_rating)
        # Get the corresponding value from ratings_list
        final_prediction = ratings_list[rounded_rating]
        predictions.append(final_prediction)
    for i in range(len(target)):
        actual_target = target[i]
        actual_targets.append(ratings_list[actual_target])
    return user_id_list, item_id_list, expected_rmse, mae, tmp_pred, rounded_ratings, predictions, target, actual_targets

In [9]:
# 1. First load your dataset
dir_data = './data/Epinions/dataset_Epinions_train80val10test10'
path_data = dir_data + ".pickle"
data_file = open(path_data, 'rb')
# history_u_lists, history_ur_lists, history_v_lists, history_vr_lists, train_u, train_v, train_r, test_u, test_v, test_r, social_adj_lists, ratings_list = pickle.load(data_file)
history_u_lists, history_ur_lists, history_v_lists, history_vr_lists, train_u, train_v, train_r, val_u, val_v, val_r, test_u, test_v, test_r, social_adj_lists, ratings_list, history_timestamp_lists = pickle.load(data_file)
testset = torch.utils.data.TensorDataset(torch.LongTensor(test_u), torch.LongTensor(test_v),torch.FloatTensor(test_r))
test_loader = torch.utils.data.DataLoader(testset, batch_size=1000, shuffle=True)

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
use_cuda = False
if torch.cuda.is_available():
	use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")

# 2. Get the dimensions
num_users = len(history_u_lists)
num_items = len(history_v_lists)
num_ratings = len(ratings_list)
print(f'User num:{num_users}')
print(f'Item  num:{num_items}')
print(f'Rating list num:{num_ratings}')
embed_dim = 64  # Should match your training configuration

# 3. Initialize and load the model
best_model_path = './checkpoints/best_model.pt'
model, epoch, rmse, mae = initialize_and_load_model(
    best_model_path,
    num_users,
    num_items,
    num_ratings,
    embed_dim,
    history_u_lists,
    history_ur_lists,
    history_v_lists,
    history_vr_lists,
    social_adj_lists,
    device
)

print(f"Loaded model from epoch {epoch} with RMSE: {rmse:.4f}, MAE: {mae:.4f}")
# 4. inference model
userID_list, itemID_list, expected_rmse, mae, predicted_ratings_continuous, predicted_ratings_round, predicted_ratings, key_ratings, value_ratings = test(model, device, test_loader, ratings_list)
# create csv result
create_result(userID_list, itemID_list, predicted_ratings_continuous, predicted_ratings_round, predicted_ratings, key_ratings, value_ratings,expected_rmse,mae)

print(f"Expected RMSE: {expected_rmse:.4f}, MAE: {mae:.4f}")

/tmp/ipykernel_2336761/2119905480.py:42: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)


User num:22167
Item  num:296278
Rating list num:6
Loaded model from epoch 10 with RMSE: 0.8246, MAE: 0.5776


Testing: 100%|██████████| 93/93 [00:33<00:00,  2.76it/s]


Metrics exported to result/metrics.txt
Data exported to result/predictions.csv
Expected RMSE: 0.8218, MAE: 0.5762
